In [ ]:
# Passo 1: Importar as bibliotecas necessárias

In [ ]:
# Definir os caminhos

# Carregar os dados

In [ ]:
# Preparar os dados

# Dividir os dados

# Treinar o classificador

In [ ]:
# Avaliar o modelo

# Salvar o modelo treinado

In [ ]:
# Definir o caminho para o modelo salvo e para os novos dados

# Carregar o modelo treinado

# Carregar e preparar a nova nuvem de pontos

# Define os nomes das colunas conforme o cabeçalho informado

# Selecionar somente as colunas relevantes para a classificação

# Classificar a nova nuvem de pontos

# Adicionar as previsões ao DataFrame e salvar os resultados

In [ ]:
# Vizualizar a Nuvem de Pontos
# Carregar os dados da nuvem de pontos

In [ ]:
# Vizualizar a Nuvem de Pontos Classificada
# Extrair as classificações

